In [1]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import plot_roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier


In [3]:
df=pd.read_csv('/content/heart.csv')

In [4]:
df.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

# Let's apply different model without taking care of categorical variable

In [6]:
X=df.drop(['target'],axis=1)
y=df['target']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y)

In [8]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8289473684210527

In [9]:
nvb=GaussianNB()
nvb.fit(X_train,y_train)
nvb.score(X_test,y_test)

0.7894736842105263

In [10]:
catb=CatBoostClassifier()
catb.fit(X_train,y_train)
catb.score(X_test,y_test)

Learning rate set to 0.00547
0:	learn: 0.6878594	total: 49.9ms	remaining: 49.9s
1:	learn: 0.6838117	total: 51.2ms	remaining: 25.5s
2:	learn: 0.6816077	total: 52.2ms	remaining: 17.4s
3:	learn: 0.6770761	total: 53.7ms	remaining: 13.4s
4:	learn: 0.6733991	total: 54.8ms	remaining: 10.9s
5:	learn: 0.6686518	total: 55.7ms	remaining: 9.23s
6:	learn: 0.6648078	total: 56.8ms	remaining: 8.05s
7:	learn: 0.6604641	total: 57.7ms	remaining: 7.16s
8:	learn: 0.6562613	total: 58.8ms	remaining: 6.47s
9:	learn: 0.6519664	total: 59.7ms	remaining: 5.91s
10:	learn: 0.6483111	total: 60.5ms	remaining: 5.44s
11:	learn: 0.6443861	total: 61.8ms	remaining: 5.09s
12:	learn: 0.6406448	total: 62.9ms	remaining: 4.78s
13:	learn: 0.6365398	total: 64ms	remaining: 4.51s
14:	learn: 0.6326733	total: 65ms	remaining: 4.27s
15:	learn: 0.6286560	total: 66ms	remaining: 4.06s
16:	learn: 0.6253108	total: 67ms	remaining: 3.87s
17:	learn: 0.6214338	total: 68ms	remaining: 3.71s
18:	learn: 0.6182475	total: 69ms	remaining: 3.56s
19:	l

0.7894736842105263

In [11]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
xgb.score(X_test,y_test)

0.7368421052631579

In [12]:
lgbm=LGBMClassifier()
lgbm.fit(X_train,y_train)
lgbm.score(X_test,y_test)

0.7631578947368421

In [13]:
model=KNeighborsClassifier(n_neighbors=1)
model.fit(X_train,y_train)
model.score(X_test,y_test)

#less score than others

0.5

### Above Models are around 82%(except knn) accurate but we didn't tuned them., In next series of code we'll be doing hyper-parameter tuning.

## Let's improve our KNN model which previously gave around 61% score

In [14]:
kfold=KFold(n_splits=5)
skfold=StratifiedKFold(n_splits=5,shuffle=True)
ss=ShuffleSplit(n_splits=8,train_size=.4,test_size=.3)
rs=RepeatedStratifiedKFold(n_splits=5,n_repeats=10)

In [15]:
print("KFold: ",cross_val_score(KNeighborsClassifier(),X,y,cv=kfold))
print("skfold: ",cross_val_score(KNeighborsClassifier(),X,y,cv=skfold))
print("ShuffleSplit: ",cross_val_score(KNeighborsClassifier(),X,y,cv=ss))
print("RepeatedStratifiedKFold: ",cross_val_score(KNeighborsClassifier(),X,y,cv=rs))


KFold:  [0.47540984 0.63934426 0.67213115 0.38333333 0.31666667]
skfold:  [0.70491803 0.68852459 0.52459016 0.68333333 0.56666667]
ShuffleSplit:  [0.59340659 0.63736264 0.61538462 0.56043956 0.65934066 0.65934066
 0.59340659 0.64835165]
RepeatedStratifiedKFold:  [0.59016393 0.67213115 0.72131148 0.65       0.65       0.6557377
 0.68852459 0.6557377  0.71666667 0.6        0.68852459 0.63934426
 0.63934426 0.65       0.66666667 0.63934426 0.70491803 0.63934426
 0.61666667 0.66666667 0.59016393 0.62295082 0.6557377  0.61666667
 0.65       0.45901639 0.70491803 0.72131148 0.7        0.65
 0.60655738 0.60655738 0.6557377  0.68333333 0.56666667 0.6557377
 0.72131148 0.73770492 0.61666667 0.66666667 0.57377049 0.6557377
 0.72131148 0.61666667 0.71666667 0.6557377  0.70491803 0.7704918
 0.68333333 0.63333333]


*KNN score is nearly 70% accurate in RepeatedStratifiedKFold*

In [16]:
from sklearn.model_selection import GridSearchCV
param_grid={'max_depth':range(1,7)}
grid=GridSearchCV(DecisionTreeClassifier(random_state=0),param_grid=param_grid,cv=10)
grid.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': range(1, 7)}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False

In [17]:
print("Best Score: ",grid.score(X_test,y_test))

Best Score:  0.6973684210526315


*Even decision tree gave us best score of 82, Let's improve our KNN model more*

# We know that KNN is distance based matric and so this time we are going to do some data preprocessing

In [18]:
data=pd.get_dummies(df,columns=['cp','fbs','restecg','exang','slope','ca','thal'])
data['trestbps']=data['trestbps']/200
data['chol']=data['chol']/564
data['thalach']=data['thalach']/564

In [19]:
data.head(5)

,age,sex,trestbps,chol,thalach,oldpeak,target,cp_0,cp_1,cp_2,cp_3,fbs_0,fbs_1,restecg_0,restecg_1,restecg_2,exang_0,exang_1,slope_0,slope_1,slope_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3
0,63,1,0.725,0.413121,0.265957,2.3,1,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0
1,37,1,0.650,0.443262,0.331560,3.5,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0
2,41,0,0.650,0.361702,0.304965,1.4,1,0,1,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0
3,56,1,0.600,0.418440,0.315603,0.8,1,0,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0
4,57,0,0.600,0.627660,0.289007,0.6,1,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0


In [20]:
df.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [21]:
data2=data.drop(['cp_0','fbs_0','restecg_0','exang_0','slope_0','ca_0','thal_0'],axis=1)

In [22]:
X=data2.drop(['target','age'],axis=1)
y=data2['target']
kfold=KFold(n_splits=5)
skfold=StratifiedKFold(n_splits=5,shuffle=True)
ss=ShuffleSplit(n_splits=8,train_size=.4,test_size=.3)
rs=RepeatedStratifiedKFold(n_splits=5,n_repeats=10)

In [23]:
print("KFold: ",cross_val_score(KNeighborsClassifier(),X,y,cv=kfold))
print("skfold: ",cross_val_score(KNeighborsClassifier(),X,y,cv=skfold))
print("ShuffleSplit: ",cross_val_score(KNeighborsClassifier(),X,y,cv=ss))
print("RepeatedStratifiedKFold: ",cross_val_score(KNeighborsClassifier(),X,y,cv=rs))

KFold:  [0.78688525 0.70491803 0.85245902 0.7        0.58333333]
skfold:  [0.85245902 0.73770492 0.7704918  0.88333333 0.83333333]
ShuffleSplit:  [0.86813187 0.79120879 0.84615385 0.82417582 0.73626374 0.8021978
 0.81318681 0.79120879]
RepeatedStratifiedKFold:  [0.80327869 0.85245902 0.80327869 0.85       0.8        0.7704918
 0.85245902 0.78688525 0.83333333 0.85       0.81967213 0.85245902
 0.85245902 0.81666667 0.8        0.85245902 0.85245902 0.83606557
 0.75       0.81666667 0.80327869 0.83606557 0.7704918  0.85
 0.83333333 0.83606557 0.81967213 0.80327869 0.81666667 0.83333333
 0.75409836 0.7704918  0.81967213 0.71666667 0.86666667 0.83606557
 0.7704918  0.81967213 0.76666667 0.83333333 0.81967213 0.81967213
 0.86885246 0.76666667 0.78333333 0.83606557 0.75409836 0.7704918
 0.88333333 0.83333333]


In [24]:
cross_val_scores=[]
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y)

for i in range(1,15,3):
  model=KNeighborsClassifier(n_neighbors=i)
  scores=cross_val_score(model, X_train,y_train,cv=10)
  cross_val_scores.append(np.mean(scores))

In [25]:
print("Best Score: ",np.max(cross_val_scores))

Best Score:  0.8019762845849803


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y)

param_grid={'n_neighbors':np.arange(1,25,3)}

gridSearch=GridSearchCV(KNeighborsClassifier(),param_grid,cv=10,return_train_score=True)

gridSearch.fit(X_train,y_train)

print("Best Score TRAIN: ",gridSearch.best_score_)
print("Best Param: ",gridSearch.best_params_)
print("Best Score Test: ",gridSearch.score(X_test,y_test))

Best Score TRAIN:  0.8335968379446639
Best Param:  {'n_neighbors': 7}
Best Score Test:  0.8289473684210527


In [31]:
from sklearn.metrics import accuracy_score

In [41]:
param_grid = [{'n_neighbors': np.arange(1, 15, 1)}]

In [42]:
kf = StratifiedKFold(n_splits=10,random_state=1,shuffle=True)
i=1
for train_index,test_index in kf.split(X,y):
     print('\n{} of kfold {}'.format(i,kf.n_splits))
     xtr,xvl = X.loc[train_index],X.loc[test_index]
     ytr,yvl = y[train_index],y[test_index]
     model = GridSearchCV(KNeighborsClassifier(), param_grid, cv=10, scoring= 'f1',iid=True)
     model.fit(xtr, ytr)
     print (model.best_params_)
     pred=model.predict(xvl)
     print('accuracy_score',accuracy_score(yvl,pred))
     i+=1


1 of kfold 10


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'n_neighbors': 7}
accuracy_score 0.8387096774193549

2 of kfold 10


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'n_neighbors': 5}
accuracy_score 0.7096774193548387

3 of kfold 10


...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'n_neighbors': 6}
accuracy_score 0.7666666666666667

9 of kfold 10


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'n_neighbors': 7}
accuracy_score 0.8666666666666667

10 of kfold 10
{'n_neighbors': 7}
accuracy_score 0.7666666666666667


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


# So we got upto 86.6% of accuracy using proper parameter


# Let's try out more robust model "XGBClassifier"

In [35]:
param_grid = [{'min_child_weight': np.arange(0.1, 10.1, 0.1)}]

In [36]:
kf = StratifiedKFold(n_splits=10,random_state=1,shuffle=True)
i=1
for train_index,test_index in kf.split(X,y):
     print('\n{} of kfold {}'.format(i,kf.n_splits))
     xtr,xvl = X.loc[train_index],X.loc[test_index]
     ytr,yvl = y[train_index],y[test_index]
     model = GridSearchCV(XGBClassifier(), param_grid, cv=10, scoring= 'f1',iid=True)
     model.fit(xtr, ytr)
     print (model.best_params_)
     pred=model.predict(xvl)
     print('accuracy_score',accuracy_score(yvl,pred))
     i+=1


1 of kfold 10


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'min_child_weight': 3.8000000000000003}
accuracy_score 0.8709677419354839

2 of kfold 10


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'min_child_weight': 5.0}
accuracy_score 0.7419354838709677

3 of kfold 10


...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'min_child_weight': 4.1}
accuracy_score 0.7333333333333333

9 of kfold 10


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


{'min_child_weight': 0.9}
accuracy_score 0.8666666666666667

10 of kfold 10
{'min_child_weight': 3.2}
accuracy_score 0.7666666666666667


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


# We got around 87% to 90% accuracy using proper parameter